In [35]:
import json
from functools import lru_cache
from typing import List

import pandas as pd
import requests
from pydantic import BaseModel, confloat, AnyUrl,conint
from ratelimit import RateLimitException, limits
from tenacity import *
from yarl import URL

In [2]:
API_ENDPOINT = URL("https://api.coingecko.com/api/v3")

In [25]:
@retry(
    retry=retry_if_exception_type(requests.exceptions.RequestException),
    wait=wait_exponential(max=60),
    stop=stop_after_attempt(3),
)
@retry(
    retry=retry_if_exception_type(RateLimitException),
    wait=wait_fixed(60),
    stop=stop_after_delay(360),
)
@limits(calls=50, period=60)
def call_get(url: URL, params: dict) -> str:
    return requests.get(url.update_query(params)).text

In [45]:
class MarketDataInfo(BaseModel):
    id: str
    symbol: str
    current_price: float
    market_cap: int
    image: AnyUrl



In [47]:
def parse_result(result) -> MarketDataInfo:
    return MarketDataInfo(
        id=result["id"],
        symbol=result["symbol"],
        current_price=result["current_price"],
        market_cap=result["market_cap"],
        image=result["image"]
    )

In [8]:
list_of_coins = pd.read_csv("/home/vlad/master_project/data/coin_list/list_of_coins.csv")['id'].to_list()

In [65]:
def get_historical_data(id: str) -> List[str]:
    return [i 
        for i in json.loads(
            call_get(API_ENDPOINT / "coins" / id / "market_chart", {"vs_currency" : "usd", "days" : "max"})
        )
    ]

In [66]:
class GetHistoricalData(BaseModel):
    timestamp: int
    price: confloat(ge=0)  
    market_caps: confloat(ge=0)
    total_volumes: conint(ge=0)


In [67]:
a = get_historical_data('bitcoin')

In [68]:
a

['prices', 'market_caps', 'total_volumes']